## 2.- Sizing Code Evolutionary Algortithm

In [255]:
import scipy
import scipy.optimize
from math import pi
from math import sqrt
import math
import timeit
import time

import numpy as np
from pyDOE import *

#### 1.- Procedure

After application of FM1, we have applied the SLSQP with a Multi-Start code to explore all the space when minimizing the objective. A different algortihm will be here applied to compare both results.
Mainly, the application of Evolutionary Algortihm permits you to explore the whole space without need of generating a DoE. It works attributing a penalization to the contraints that don't satisfy the objective.

- $n_{pro,to}D_{pro}<nD_{max}$
- $T_{moy,mot}>T_{pro,hov}$
- $T_{max,mot}>T_{pro,to}$
- $P_{esc}=P_{esc,to}$
- $U_{bat} > U_{mot,to}$   
- $U_{bat} > U_{esc}$ 
- $U_{bat}I_{bat} > \frac{U_{mot,to}I_{mot,to}N_{pro,to}}{\eta_{esc}}$
- $0.8C_{bat} = I_{bat} \cdot t_{hov}$
- $L_{arm}= \frac{D_{pro}/2}{sin(\pi/n_{arm})}$
- $\displaystyle \frac{F_{max,arm} \cdot L_{arm}}{\frac{\pi \cdot \left(D_{out}^4-D_{in}^4\right)}{32 \cdot D_{out}}}=\sigma_{max}$

Equalities will not be handled as constraints any more.

#### 2.- Problem Definition

In [256]:
# Specifications

# Load
M_load=5.3 # [kg] load mass

# Acceleration take off
a_to= 1*9.81 # [m/s²] acceleration

# Autonomy
t_h=15 # [min] time of hover fligth

# Architecture of the multi-rotor drone (4,6, 8 arms, ...)
Narm=8 # [-] number of arm
Np_arm=1 # [-] number of propeller per arm (1 or 2)
Npro=Np_arm*Narm # [-] Propellers number


#### 3.- Sizing Code

In [257]:
# -----------------------
# sizing code
# -----------------------
# inputs:
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Design variables
# ---
    k_M=param[0] # over sizing coefficient on the load mass 
    k_mot=param[1] # over sizing coefficient on the motor torque
    k_speed_mot=param[2] # over sizing coefficient on the motor speed
    k_vb=param[3] # over sizing coefficient for the battery voltage
    k_ND=param[4] # slow down propeller coef : ND = kNDmax / k_ND
    D_ratio=param[5] # aspect ratio e/c (thickness/side) for the beam of the frame
    k_Mb=param[6] # over sizing coefficient on the battery load mass 
    beta=param[7] # pitch/diameter ratio of the propeller
    

# Hover & Take-Off thrust 
# ---
    Mtotal=k_M*M_load # [kg] Estimation of the total mass (or equivalent weight of dynamic scenario)
    Tpro_hover=Mtotal*(9.81)/Npro # [N] Thrust per propeller for hover
    Tpro_takeoff=Mtotal*(9.81+a_to)/Npro # [N] Thrust per propeller for take-off

# Propeller selection
# ---
    rho_air=1.18 # [kg/m^3] air density 
    
# Propeller characteristicss
# Ref : APC 

    C_t=4.27e-02 + 1.44e-01 * beta # Thrust coef with T=C_T.rho.n^2.D^4
    C_p=-1.48e-03 + 9.72e-02 * beta  # Power coef with P=C_p.rho.n^3.D^5

    NDmax= 105000/60*.0254# [Hz.m] max speed limit (N.D max)

    Dpro_ref=11*.0254 # [m] diameter
    Mpro_ref=0.53*0.0283 # [kg] mass

# Propeller selection with take-off scenario

    
    Dpro=(Tpro_takeoff/(C_t*rho_air*(NDmax/k_ND)**2))**0.5  # [m] Propeller diameter
    n_pro_takeoff=NDmax/k_ND/Dpro # [Hz] Propeller speed 
    
    #print(k_ND)
    #print(n_pro_takeoff*60, 'RPM')
    #print(n_pro_takeoff*60*Dpro/.0254, 'RPMxInch')
    Wpro_takeoff=n_pro_takeoff*2*3.14 # [rad/s] Propeller speed
    
    Mpro=Mpro_ref*(Dpro/Dpro_ref)**3 # [kg] Propeller mass
    
    Ppro_takeoff=C_p*rho_air*n_pro_takeoff**3*Dpro**5# [W] Power per propeller
    Qpro_takeoff=Ppro_takeoff/Wpro_takeoff # [N.m] Propeller torque
    #print(Qpro , 'N.m')

# Propeller torque& speed for hover

    n_pro_hover=sqrt(Tpro_hover/(C_t*rho_air*Dpro**4)) # [Hz] hover speed
    Wpro_hover=n_pro_hover*2*3.14 # [rad/s] Propeller speed    
    
    Ppro_hover=C_p*rho_air*n_pro_hover**3*Dpro**5# [W] Power per propeller
    Qpro_hover=Ppro_hover/Wpro_hover # [N.m] Propeller torque    

    #Pesc_ref=3108 # [W] Power
    #Vesc_ref=44.4 #[V]Voltage

    V_bat_est=k_vb*1.84*(Ppro_takeoff)**(0.36) # [V] battery voltage estimation
    #V_bat_est=k_vb*Vesc_ref*(Ppro_takeoff/Pesc_ref)**(1/3) # [V] battery voltage estimation

# Motor selection & scaling laws
# ---
    # Motor reference
    # Ref : AXI 5325/16 GOLD LINE
    Tmot=k_mot*Qpro_hover   # [N.m] Motor nominal torque per propeller

    Tmot_ref=2.32  # [N.m] rated torque
    Tmot_max_ref=85/70*Tmot_ref # [N.m] max torque
    Rmot_ref=0.03  # [Ohm] resistance
    Mmot_ref=0.575 # [kg] mass
    Ktmot_ref=0.03 # [N.m/A] torque coefficient
    Tfmot_ref=0.03 # [N.m] friction torque (zero load, nominal speed) 
    
    Mmot=Mmot_ref*(Tmot/Tmot_ref)**(3/3.5) # [kg] Motor mass
    Tmot_max=Tmot_max_ref*(Tmot/Tmot_ref)**(1) # [N.m] max torque
    
    # Selection with take-off speed
    Ktmot=V_bat_est/(k_speed_mot*Wpro_takeoff) # [N.m/A] or [V/(rad/s)] Kt motor (RI term is missing)
    Rmot=Rmot_ref*(Tmot/Tmot_ref)**(-5/3.5)*(Ktmot/Ktmot_ref)**(2)  # [Ohm] motor resistance
    Tfmot=Tfmot_ref*(Tmot/Tmot_ref)**(3/3.5) # [N.m] Friction torque

    # Hover current and voltage
    Imot_hover = (Qpro_hover+Tfmot)/Ktmot # [I] Current of the motor per propeller
    Umot_hover = Rmot*Imot_hover + Wpro_hover*Ktmot # [V] Voltage of the motor per propeller
    P_el_hover = Umot_hover*Imot_hover # [W] Hover : output electrical power
    
    # Take-Off current and voltage
    Imot_takeoff = (Qpro_takeoff+Tfmot)/Ktmot # [I] Current of the motor per propeller
    Umot_takeoff = Rmot*Imot_takeoff + Wpro_takeoff*Ktmot # [V] Voltage of the motor per propeller
    P_el_takeoff=Umot_takeoff*Imot_takeoff # [W] Takeoff : output electrical power
    
   
# Battery selection & scaling laws  
# --- 
    # Battery
    # Ref : MK-quadro
    Mbat_ref=.329 # [kg] mass
    #Ebat_ref=4*3.7*3.3*3600 # [J] energy
#    Ebat_ref=220*3600*.329 # [J]
    Cbat_ref= 3.400*3600#[A.s] 
    Vbat_ref=4*3.7#[V] 
    Imax_ref=170#[A]

    Ncel=V_bat_est/3.7# [-] Cell number, round (up value)
    V_bat=3.7*Ncel # [V] Battery voltage
    
    Mbat=k_Mb*M_load # Battery mass
    
    # Hover --> autonomy
    C_bat = Mbat/Mbat_ref*Cbat_ref/V_bat*Vbat_ref # [A.s] Capacity  of the battery 
    I_bat = (P_el_hover*Npro)/.95/V_bat # [I] Current of the battery
    t_hf = .8*C_bat/I_bat/60 # [min] Hover time 
    Imax=Imax_ref*C_bat/Cbat_ref
    

# ESC
# Ref : Turnigy K_Force 70HV 
    Pesc_ref=3108 # [W] Power
    Vesc_ref=44.4 #[V]Voltage
    Mesc_ref=.115 # [kg] Mass
    
    P_esc=P_el_takeoff*V_bat/Umot_takeoff # [W] power electronic power (corner power or apparent power)
    
    Mesc = Mesc_ref*(P_esc/Pesc_ref) # [kg] Mass ESC
    Vesc=Vesc_ref*(P_esc/Pesc_ref)**(1/3)# [V] ESC voltage
    
# Frame
# ---

    Mfra_ref=.347 #[kg] MK7 frame
    Marm_ref=0.14#[kg] Mass of all arms

    # Length calculation   
#    sep= 2*pi/Narm #[rad] interior angle separation between propellers
    Lb=Dpro/2/(math.sin(pi/Narm))  #[m] length of the arm

    # Static stress
    # Sigma_max=200e6/4 # [Pa] Alu max stress (2 reduction for dynamic, 2 reduction for stress concentration)
    Sigma_max=280e6/4 # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
    
    # Tube diameter & thickness
    Dout=(Tpro_takeoff*Lb*32/(pi*Sigma_max*(1-D_ratio**4)))**(1/3) # [m] outer diameter of the beam 
    D_ratio # [m] inner diameter of the beam

    # Mass
    Marm=pi/4*(Dout**2-(D_ratio*Dout)**2)*Lb*1700*Narm # [kg] mass of the arms

    Mfra=Mfra_ref*(Marm/Marm_ref)# [kg] mass of the frame

    # Objective and Constraints sum up
    # ---
    Mtotal_final = (Mesc+Mpro+Mmot)*Npro+M_load+Mbat+Mfra+Marm

    constraints = [(Mtotal-Mtotal_final)/Mtotal_final,(Tmot_max-Qpro_takeoff)/Qpro_takeoff,(V_bat-Umot_takeoff)/V_bat,(V_bat-Vesc)/V_bat,(V_bat*Imax-Umot_takeoff*Imot_takeoff*Npro/0.95)/(V_bat*I_bat),(t_hf-t_h)/t_hf] 
    
# Objective and contraints
    if arg=='Obj':
        P=0 # Penalisation nulle
        for C in constraints: 
            if (C<0.): 
                P=P-1e9*C
        return Mtotal_final+P # for mass optimisation       
            
        
    elif arg=='Prt':
        print("* Specifications:")
        print("           Mass of load : %.2f kg"% M_load)
        print("           Take off acceleration : %.2f g"%(a_to/9.81))
        print("* Optimisation objective:")
        print("           Max Autonomy : %.3f min"%t_hf)
        print("           Total mass : %.3f kg"%Mtotal_final)   
        print("* Optimisation variables:")
        print("           beta angle consisting of pitch /diameter = ", beta)
        print("           oversizing coefficient on the load mass k_M = %.2f"% k_M)
        print("           Ratio for battery mass, k_Mb = %.2f"%k_Mb)
        print("           oversizing coefficient on the motor torque k_mot = %.2f"%k_mot)
        print("           oversizing coefficient on the motor speed k_speed_mot = %.2f"%k_speed_mot)
        print("           undersizing coefficient on the propeller speed k_ND = %.2f"%(k_ND))
        print("           aspect ratio inner diam/ outer diam = %.2f"%D_ratio)
        print("           over sizing coefficient on the battery load mass = %.2f"%k_Mb)
        print("           over sizing coefficient for the battery voltage = %.2f"%k_vb) 
        print("* Architecture description:")
        print("           Numbers of arms = ",Narm)        
        print("           Numbers of propellers = ",Npro)        
        print("")
        print("* Mass values:")
        print("           Propeller mass (1x): %.3f kg"%(Mpro))
        print("           Motor mass (1x): %.3f kg"%(Mmot))
        print("           Battery mass: %.3f kg"%(Mbat))
        print("           ESC mass per propeller : %.3f kg"%(Mesc))
        print("           Arm mass total : %.3f kg"%(Marm))
        print("           Frame mass total : %.3f kg"%(Mfra))
        print("")
        print("* Frame size:")
        print("           Beam diameter = %.2f mm" %(Dout*1000))
        print("           Inner diameter = %.2f mm" % (D_ratio*Dout*1000))
        print("           Length of the arm = %.2f mm" % (Lb*1000))
        print("")
        print("* Propellers:")
        print("           NxD takeoff = %.0f RPMxInch"%(n_pro_takeoff*60*Dpro/.0254))
        print("           Diameter Dpro = %.2g m"%Dpro)
        print("           Pitch  = %.2g m"%(beta*Dpro))
#        print("           Ratio of useful area = %.2f %%"%(k_surf_real*100))
        print("           Power coefficient C_p: - for statics: %.4f "%(C_p))
        print("")
        print("           Thrust coefficient C_t: - for statics: %.4f "%(C_t))
        print("")
        print("           Rotational speed:      - for hover: %.0f RPM "%(Wpro_hover*60/2/pi))
        print("                                  - for vertical acceleration: %.0f RPM "%(Wpro_takeoff*60/2/pi))
        print("")
        print("* Thrust force per propeller:")
        print("           Hover: %.2f N"%(Tpro_hover))
        print("           Vertical + Acceleration (2g): %.2f  N"%(Tpro_takeoff))
        print("")
        print("* Power per propeller:")
        print("           Hover power: %.2f W"%(Ppro_hover))
        print("           Vertical + Acceleration (2g): %.2f  W"%(Ppro_takeoff))
        print("")
        print("* Torque per propeller:")
        print("           Hover : %.3f N.m"%(Qpro_hover+Tfmot))
        print("           Vertical + Acceleration (2g): %.3f  N.m"%(Qpro_takeoff+Tfmot))
        print("")
        print("* Voltage:")
        print("           Motor voltage Umot: %.2f V"%(Umot_hover))
        print("           Transient voltage in vertical + Acceleration (2g): U_ver: %.2f V"%(Umot_takeoff))
        print("           Kt motor Kt: %.2f N.m/A"%(Ktmot))        
        print("")
        print("* Current:")
        print("           Motor current Hover: %.2f A"%(Imot_hover))
        print("           Transient current in vertical + Acceleration (2g): I_ver: %.2f A"%(Imot_takeoff))
        print("")
        print("*ESC:")
        print("           Power ESC: %.2f W"%(P_esc))
        print("")
        print("* Battery:")
        print("           Battery capacity = %.2f A.h" %(C_bat/3600))
        print("           Battery voltage = %.2f V" %(V_bat))
        print("           ESC voltage = %.2f V" %(Vesc))
        print("           Battery current = %.2f A" %(I_bat))        
        print("           Maximum current = %.2f A" %(Imax))        
        print("")      
        print("* Normalized constraints (should be >0):")
        print("           (Mtotal-Mtotal_final) =%.2f  " %constraints[0])
        print("           (Tmot_max-Qpro_takeoff)/Qpro_takeoff= %.3f"%constraints[1])
        print("           (V_bat-Umot_takeoff)/V_bat = %.3f "%constraints[2])
        print("           (V_bat-Vesc)/V_bat = %.3f "%constraints[3])
        print("           (V_bat*Imax-Umot_takeoff*Imot_takeoffr*Npro/0.95)/(vbat*I_bat)= %.3f "%constraints[4])
        print("           (t_hf-t_h)/t_hf = %.3f "%constraints[5])
    else:
        return constraints


### 4.-Optimization variables

### 5.-Result

In [258]:
# optimization with SLSQP algorithm
contrainte=lambda x: SizingCode(x, 'Const')
objectif=lambda x: SizingCode(x, 'Obj')

# Differential evolution omptimisation
start = time.time()
result = scipy.optimize.differential_evolution(func=objectif,
                               bounds=[(1,20),#k_M
                                       (1,10),#k_mot
                                       (1,10),#k_speed_mot
                                       (1,5),#k_vb
                                       (1,10),#k_ND
                                       (0.05,.99),#D_ratio
                                       (.01,15),#k_Mb
                                       (0.3,0.6),#beta
                                    ],
                               tol=1e-12)
# Final characteristics after optimization 
end = time.time()
print("Operation time: %.5f s" %(end - start))

print("-----------------------------------------------")
print("Final characteristics after optimization :")

SizingCode(result.x, 'Prt')
print("-----------------------------------------------")


Operation time: 7.73525 s
-----------------------------------------------
Final characteristics after optimization :
* Specifications:
           Mass of load : 5.30 kg
           Take off acceleration : 1.00 g
* Optimisation objective:
           Max Autonomy : 15.000 min
           Total mass : 9.276 kg
* Optimisation variables:
           beta angle consisting of pitch /diameter =  0.30000007981685767
           oversizing coefficient on the load mass k_M = 1.75
           Ratio for battery mass, k_Mb = 0.46
           oversizing coefficient on the motor torque k_mot = 1.65
           oversizing coefficient on the motor speed k_speed_mot = 1.27
           undersizing coefficient on the propeller speed k_ND = 1.00
           aspect ratio inner diam/ outer diam = 0.99
           over sizing coefficient on the battery load mass = 0.46
           over sizing coefficient for the battery voltage = 2.30
* Architecture description:
           Numbers of arms =  8
           Numbers of prope